<a href="https://colab.research.google.com/github/dalismo/Finance_News_Sentiment/blob/ai%2Fworking_on_count_vector/AI_v3_count_vector_ml_sentiment_analysis_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("StopWords").getOrCreate()
spark = SparkSession.builder.appName("Fin-sent-count-vector").getOrCreate()

In [3]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles

url = 'https://raw.githubusercontent.com/dalismo/Finance_News_Sentiment/main/Resources/sentiment-train.csv'
# df = pd.read_csv(url,encoding='latin-1', header=None)
# url ="https://s3.amazonaws.com/dataviz-curriculum/day_2/yelp_reviews.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("sentiment-train.csv"), sep=",", header=None)

# Show DataFrame
df.show(5, truncate=False)

+--------+-----------------------------------------------------------------------------------------------------+
|_c0     |_c1                                                                                                  |
+--------+-----------------------------------------------------------------------------------------------------+
|Positive|Autodesk Is Down 15% After Q2 Earnings -- Is This a Buying Opportunity?                              |
|Positive|Blue Shield of California, Walgreens Partner on Personalized Healthcare Solution                     |
|Positive|RSX: Russian Equities Are Still Undervalued                                                          |
|Positive|XLB: Hold Your Position                                                                              |
|Positive|Musgrave Minerals broad, near-surface high-grade results open up further Cue gold discovery potential|
+--------+--------------------------------------------------------------------------------------

In [5]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df = df.withColumn('length', length(df['_c1']))
data_df.show()

+--------+--------------------+------+
|     _c0|                 _c1|length|
+--------+--------------------+------+
|Positive|Autodesk Is Down ...|    71|
|Positive|Blue Shield of Ca...|    80|
|Positive|RSX: Russian Equi...|    43|
|Positive|XLB: Hold Your Po...|    23|
|Positive|Musgrave Minerals...|   101|
|Positive|ViacomCBS to rest...|    49|
|Positive|Oil climbs to one...|    51|
|Negative|Australian regula...|    67|
|Positive|The Wall Street J...|    90|
|Positive|Ora Banda Mining ...|    50|
|Positive|Google Pay Plans ...|    43|
|Negative|Toyota, Honda Ang...|    69|
| Neutral|ZY Investor Alert...|    44|
| Neutral|Calima Energy spu...|    58|
|Positive|These Marijuana S...|    56|
|Positive|Moderna ETFs to R...|    61|
| Neutral|MacroGenics Annou...|    72|
| Neutral|Zymeworks Announc...|   173|
| Neutral|BioXcel Therapeut...|   130|
| Neutral|Merck to Present ...|    80|
+--------+--------------------+------+
only showing top 20 rows



In [6]:
df2 = data_df.withColumnRenamed("_c0", "class").withColumnRenamed("_c1", "text")
df2.printSchema()
df2.show(20, truncate = False)


root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)

+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|class   |text                                                                                                                                                                         |length|
+--------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------+
|Positive|Autodesk Is Down 15% After Q2 Earnings -- Is This a Buying Opportunity?                                                                                                      |71    |
|Positive|Blue Shield of California, Walgreens Partner on Personalized Healthcare Solution                        

In [7]:
df2.createOrReplaceTempView('temp')
df3 = spark.sql('select case class when "Positive" then 1.0  when "Negative" then 2.0 else 0 end as label, text from temp')
df3.show(20, truncate = False)

+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|text                                                                                                                                                                         |
+-----+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1.0  |Autodesk Is Down 15% After Q2 Earnings -- Is This a Buying Opportunity?                                                                                                      |
|1.0  |Blue Shield of California, Walgreens Partner on Personalized Healthcare Solution                                                                                             |
|1.0  |RSX: Russian Equities Are Still Undervalued                                        

In [8]:
from pyspark.ml.feature import  Tokenizer
tokenizer = Tokenizer(inputCol="text", outputCol="words")
wordsData = tokenizer.transform(df3)
wordsData.show()

+-----+--------------------+--------------------+
|label|                text|               words|
+-----+--------------------+--------------------+
|  1.0|Autodesk Is Down ...|[autodesk, is, do...|
|  1.0|Blue Shield of Ca...|[blue, shield, of...|
|  1.0|RSX: Russian Equi...|[rsx:, russian, e...|
|  1.0|XLB: Hold Your Po...|[xlb:, hold, your...|
|  1.0|Musgrave Minerals...|[musgrave, minera...|
|  1.0|ViacomCBS to rest...|[viacomcbs, to, r...|
|  1.0|Oil climbs to one...|[oil, climbs, to,...|
|  2.0|Australian regula...|[australian, regu...|
|  1.0|The Wall Street J...|[the, wall, stree...|
|  1.0|Ora Banda Mining ...|[ora, banda, mini...|
|  1.0|Google Pay Plans ...|[google, pay, pla...|
|  2.0|Toyota, Honda Ang...|[toyota,, honda, ...|
|  0.0|ZY Investor Alert...|[zy, investor, al...|
|  0.0|Calima Energy spu...|[calima, energy, ...|
|  1.0|These Marijuana S...|[these, marijuana...|
|  1.0|Moderna ETFs to R...|[moderna, etfs, t...|
|  0.0|MacroGenics Annou...|[macrogenics, ann...|


In [9]:
from pyspark.ml.feature import CountVectorizer
count = CountVectorizer (inputCol="words", outputCol="rawFeatures")
model = count.fit(wordsData)
featurizedData = model.transform(wordsData)
featurizedData.show()

+-----+--------------------+--------------------+--------------------+
|label|                text|               words|         rawFeatures|
+-----+--------------------+--------------------+--------------------+
|  1.0|Autodesk Is Down ...|[autodesk, is, do...|(11788,[6,9,19,37...|
|  1.0|Blue Shield of Ca...|[blue, shield, of...|(11788,[2,7,177,5...|
|  1.0|RSX: Russian Equi...|[rsx:, russian, e...|(11788,[48,262,63...|
|  1.0|XLB: Hold Your Po...|[xlb:, hold, your...|(11788,[158,335,1...|
|  1.0|Musgrave Minerals...|[musgrave, minera...|(11788,[26,49,89,...|
|  1.0|ViacomCBS to rest...|[viacomcbs, to, r...|(11788,[0,34,1891...|
|  1.0|Oil climbs to one...|[oil, climbs, to,...|(11788,[0,7,100,1...|
|  2.0|Australian regula...|[australian, regu...|(11788,[5,175,179...|
|  1.0|The Wall Street J...|[the, wall, stree...|(11788,[1,7,292,4...|
|  1.0|Ora Banda Mining ...|[ora, banda, mini...|(11788,[0,5,72,32...|
|  1.0|Google Pay Plans ...|[google, pay, pla...|(11788,[75,217,26...|
|  2.0

In [10]:
from pyspark.ml.feature import  IDF
#IDF down-weighs features which appear frequently in a corpus. This generally improves performance when using text as features since most frequent, and hence less important words, get down-weighed.
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
rescaledData.select("label", "features").show()  
# We want only the label and features columns for our machine learning models

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  1.0|(11788,[6,9,19,37...|
|  1.0|(11788,[2,7,177,5...|
|  1.0|(11788,[48,262,63...|
|  1.0|(11788,[158,335,1...|
|  1.0|(11788,[26,49,89,...|
|  1.0|(11788,[0,34,1891...|
|  1.0|(11788,[0,7,100,1...|
|  2.0|(11788,[5,175,179...|
|  1.0|(11788,[1,7,292,4...|
|  1.0|(11788,[0,5,72,32...|
|  1.0|(11788,[75,217,26...|
|  2.0|(11788,[2,65,230,...|
|  0.0|(11788,[43,44,63,...|
|  0.0|(11788,[10,83,338...|
|  1.0|(11788,[0,3,13,18...|
|  1.0|(11788,[0,7,172,2...|
|  0.0|(11788,[11,12,13,...|
|  0.0|(11788,[1,3,5,11,...|
|  0.0|(11788,[0,2,3,12,...|
|  0.0|(11788,[0,4,12,13...|
+-----+--------------------+
only showing top 20 rows



In [11]:
seed = 0  # set seed for reproducibility
trainDF, testDF = rescaledData.randomSplit([0.8,0.2],seed)

In [12]:
trainDF.count()

3965

In [13]:
testDF.count()

1035

In [22]:
#Logistic Regression Classifier
#Cross validator different classification to  
#Param Grid apply different parameters, testing which param is best
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import numpy as np
lr = LogisticRegression(maxIter = 10)

paramGrid_lr = ParamGridBuilder() \
    .addGrid(lr.regParam, np.linspace(0.3, 0.01, 10)) \
    .addGrid(lr.elasticNetParam, np.linspace(0.3, 0.8, 6)) \
    .build()
crossval_lr = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid_lr,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds= 5)  
cvModel_lr = crossval_lr.fit(trainDF)
best_model_lr = cvModel_lr.bestModel.summary
best_model_lr.predictions.columns

['label',
 'text',
 'words',
 'rawFeatures',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [23]:
#Area under the curve for the training data
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval_lr = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
my_eval_lr.evaluate(best_model_lr.predictions)

0.9950498740046572

In [24]:
#F1 score
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
my_mc_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
my_mc_lr.evaluate(best_model_lr.predictions)

0.9931852586669052

In [25]:
#Accuracy 
my_mc_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
my_mc_lr.evaluate(best_model_lr.predictions)

0.9931904161412358

In [27]:
#prediction
train_fit_lr = best_model_lr.predictions.select('label','prediction')
train_fit_lr.groupBy('label','prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  1.0|       1.0| 1515|
|  0.0|       1.0|    4|
|  1.0|       0.0|   13|
|  2.0|       2.0|  448|
|  2.0|       1.0|    8|
|  0.0|       0.0| 1975|
|  0.0|       2.0|    2|
+-----+----------+-----+



In [29]:
#Predict using the test data and evaluate the predictions
predictions_lr = cvModel_lr.transform(testDF)
predictions_lr.columns

['label',
 'text',
 'words',
 'rawFeatures',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [30]:
predictions_lr.show(5)

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|                text|               words|         rawFeatures|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  0.0|180 Life Sciences...|[180, life, scien...|(11788,[10,12,197...|(11788,[10,12,197...|[1.25814392677280...|[0.50740231372577...|       0.0|
|  0.0|1847 Goedeker Iss...|[1847, goedeker, ...|(11788,[45,87,474...|(11788,[45,87,474...|[1.47435867402671...|[0.70263487773694...|       0.0|
|  0.0|3 Reasons to Buy ...|[3, reasons, to, ...|(11788,[0,4,23,54...|(11788,[0,4,23,54...|[0.57562822157586...|[0.11961278175929...|       1.0|
|  0.0|3 Things to Watch...|[3, things, to, w...|(11788,[0,1,3,8,3...|(11788,[0,1,3,8,3...|[0.12423249346035...|[0.19172097468053.

In [31]:
#Show sample predictions:
predictions_lr.select('label', 'prediction').show(5)

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  0.0|       0.0|
+-----+----------+
only showing top 5 rows



In [32]:
#hit vs missed
predictions_lr.groupBy('label','prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  2.0|       1.0|   30|
|  0.0|       1.0|   69|
|  2.0|       2.0|   63|
|  1.0|       1.0|  309|
|  2.0|       0.0|   16|
|  0.0|       0.0|  439|
|  0.0|       2.0|    7|
|  1.0|       2.0|    6|
|  1.0|       0.0|   96|
+-----+----------+-----+



In [33]:
#Area under the curve with the test data
my_eval_lr.evaluate(predictions_lr)

0.8229798356982823

In [34]:
#F1-score with the test data
my_mc_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
my_mc_lr.evaluate(predictions_lr)

0.7812021159635156

In [35]:
#Acccuracy with the test data
my_mc_lr = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
my_mc_lr.evaluate(predictions_lr)

0.7835748792270532

In [37]:
#Naive Bayes using MulticlassClassificationEval
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
nb = NaiveBayes()
paramGrid_nb = ParamGridBuilder() \
    .addGrid(nb.smoothing, np.linspace(0.3, 10, 10)) \
    .build()
crossval_nb = CrossValidator(estimator=nb,
                          estimatorParamMaps=paramGrid_nb,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds= 5) 
cvModel_nb = crossval_nb.fit(trainDF)

In [38]:
#average area under the curve values
cvModel_nb.avgMetrics

[0.7214073184788686,
 0.7259471433689856,
 0.7407036452208704,
 0.7499020425017013,
 0.7551169678545439,
 0.7610649146632533,
 0.7649363041198762,
 0.7658491742282761,
 0.7670193633569241,
 0.7691465583951299]

In [39]:
#Make predictions:
predictions_nb = cvModel_nb.transform(testDF)
predictions_nb.select('label', 'prediction').show(30)

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       2.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       2.0|
|  0.0|       0.0|
|  0.0|       0.0|
+-----+----------+
only showing top 30 rows



In [40]:
#hit vs missed
predictions_nb.groupBy('label','prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  2.0|       1.0|   35|
|  0.0|       1.0|  106|
|  2.0|       2.0|   71|
|  1.0|       1.0|  349|
|  2.0|       0.0|    3|
|  0.0|       0.0|  374|
|  1.0|       2.0|   17|
|  0.0|       2.0|   35|
|  1.0|       0.0|   45|
+-----+----------+-----+



In [41]:
#Area under ROC
my_eval_nb = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
my_eval_nb.evaluate(predictions_nb)

0.8092755787901419

In [42]:
#F1
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
my_mc_nb = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='f1')
my_mc_nb.evaluate(predictions_nb)

0.7693098269973866

In [43]:
#Accuracy
my_mc_nb = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='label', metricName='accuracy')
my_mc_nb.evaluate(predictions_nb)

0.7671497584541063